# TC-ADD-0041 - Weather-Induced Extremes Digital Twin data access with EODAG

This Notebook was prepared to allow the testing the usage of the EODAG Python Client to access data from the Weather-Induced Extremes Digital Twin.

The key features assessed are the following:
- Existence of DEDL as a provider in EODAG’s list of providers
- Possibility to use EODAG to discover the EO.ECMWF.DAT.DT_EXTREMES provided by DEDL
- Search for items inside of it

The Notebook actions results can be found in the generated TC-ADD-0041-report, which contains a detailed account of the success/failure of each step.

## Report Generation

The report is generated through the usage of [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/), which is a Python library for pulling data out of HTML and XML files. It contains tools that facilitate navigating, searching, and modifying the content of these types of files. The following cell creates the base HTML file to which entries will be added after the execution of key test steps. The function entryWrite() is also defined here, which will be used to edit the base HTML file by adding an entry for each step that needs to be recorded with the result of the action. 

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup("""<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8"/>
        <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
        <title>
            TC-ADD-0041 Test Report
        </title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                display: flex;
                flex-direction: column;
                justify-content: center;
                align-items: center;
            }
            
            header {
                background-color: #333;
                color: #fff;
                padding: 20px;
                width: 100%;
                display: flex;
                justify-content: center;
                align-items: center;
            }
            
            h1 {
                margin: 0;

            }
            
            table {
                border-collapse: collapse;
                margin: 20px;
                width: 80%;
            }
            
            th, td {
                border: 1px solid #ddd;
                padding: 10px;
                text-align: center;
            }
            
            .TextBoxClass {
                text-align: left;
                max-width: 20vw;
                overflow-wrap: anywhere;
            }

            .pass {
                color: green;
            }

            .fail {
                color: red;
            }

            th {
                background-color: #f2f2f2;
            }
            
            tr:nth-child(even) {
                background-color: #f2f2f2;
            }
        </style>
    </head>
    <body>
        <header>
            <h1>TC-ADD-0041 Test Report</h1>
        </header>
        <table>
            <thead>
                <tr>
                    <th>Command/Request</th>
                    <th>Expected Result</th>
                    <th>Action Result</th>
                    <th>Pass/Fail</th>
                </tr>
            </thead>
            <tbody>
            </tbody>
        </table>
    </body>
</html>""", 'html.parser')

def entryWrite(command: str, expected_result: str, result: str, pass_fail: str):
    
    new_entry = BeautifulSoup("""
    <tr>
        <td id="command" class="TextBoxClass">placeholder</td>
        <td id="expected_result" class="TextBoxClass">placeholder</td>
        <td id="result" class="TextBoxClass">placeholder</td>
        <td id="pass_fail" class="placeholder">placeholder</td>
    </tr>""", "html.parser")


    command_tag = new_entry.tr.select_one("#command")
    command_tag.string = command

    expected_result_tag = new_entry.tr.select_one("#expected_result")
    expected_result_tag.string = expected_result

    result_tag = new_entry.tr.select_one("#result")
    result_tag.string = result


    passFailTag = new_entry.tr.select_one("#pass_fail")
    passFailTag.string = pass_fail
    passFailTag["class"] = "pass" if pass_fail == "PASS" else "fail"

    soup.body.table.tbody.append(new_entry)

    with open("TC-ADD-0041-report.html", "w") as report:
        report.write(BeautifulSoup.prettify(soup))

    report.close()

# Initial setup of EODAG

The following cell allows the configuration of EODAG, necessary for a correct execution.

In [ ]:
from getpass import getpass
from datetime import datetime,timedelta

DESP_USERNAME = input("Please input your DESP username: ")
DESP_PASSWORD = getpass("Please input your DESP password: ")

dedl_config = \
f"""
dedl:
    priority: 10
    search:
        timeout: 60
    download:
        outputs_prefix: /home/jovyan/Notebook_Validation
    auth:
        credentials:
            username: {DESP_USERNAME}
            password: {DESP_PASSWORD}
"""

# DEDL As An Available Provider

The following cell contains code that checks the existence of DEDL as an available provider according to DEDL. This is necessary to allow the data search and download performed in the next steps.

In [ ]:
import os
from eodag import EODataAccessGateway

dag = EODataAccessGateway()

dag.update_providers_config(yaml_conf=dedl_config)

providers = dag.available_providers()

entryWrite(
    "dag.available_providers()",
    "A list of all EODAG providers, which includes dedl",
    ", ".join(providers),
    "PASS" if ("dedl" in providers) else "FAIL"
)

print("Entry added to the report")

# List DEDL's Product Types

The execution of this cell will have two objectives:
- Get the list of product types (collections) available to the EODAG user when specifying DEDL as a provider;
- Verify the presence of the EO.ECMWF.DAT.DT_EXTREMES in the results;

In [ ]:
collection_list = dag.list_product_types(provider="dedl", fetch_providers=False)

def idGetter(col):
    return col["ID"]

id_list = [idGetter(collection) for collection in collection_list]

entryWrite(
    'dag.list_product_types(provider="dedl")',
    "The returned list contains the Climate DT dataset",
    "Id's of the datasets that were found: " + (", ".join(id_list) if (len(id_list) != 0 ) else "None"),
    "PASS" if ("EO.ECMWF.DAT.DT_EXTREMES" in id_list) else "FAIL"
)


print("Entry added to the report")
print("List of collections found: ")
id_list

# Using AVISO to find a valid datetime

The AVISO tool is leveraged here to find a valid datetime for the download request. The Extremes DT dataset is characterized by having a rolling buffer of data from the past 15 days, although it is not guaranteed that data was produced every day. AVISO allows us to find the right date. The next cell will install the AVISO Python package.

In [ ]:
import sys
!{sys.executable} -m pip install pyaviso --quiet

## AVISO imports and declaration of configuration variables

In [ ]:
from pyaviso import NotificationManager, user_config

LISTENER_EVENT = "data"  # Event for the listener, options are mars and dissemination
TRIGGER_TYPE = "function"  # Type of trigger for the listener

REQUEST = {
    "class": "d1",
    "expver": "0001",
    "stream": "oper",
    "type": "fc",
    "time": "00",
    "step": "0",
    "levtype": "sfc",
}  # Request configuration for the listener


CONFIG = {
    "notification_engine": {
        "host": "aviso.lumi.apps.dte.destination-earth.eu",
        "port": 443,
        "https": True,
    },
    "configuration_engine": {
        "host": "aviso.lumi.apps.dte.destination-earth.eu",
        "port": 443,
        "https": True,
    },
    "schema_parser": "generic",
    "remote_schema": True,
    "auth_type": "none",
    "quiet" : True
}  # manually defined configuration

START_DATE=datetime.now() - timedelta(days=15)
END_DATE=datetime.now()

# This variable will be set by the listener's handler function with the most recent vaild date
valid_date=""

## Event handler
When the listener receives a notification, this function will be triggered. Besides printing the dates of existing data, it will also set the *valid_date* variable with the most recent valid date.

In [ ]:
def triggered_function(notification):
    """
    Function for the listener to trigger.
    """
    
    #pp(notification)
    # Access the date field
    date_str = notification['request']['date']    

    # Convert the date string to a datetime object
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    formatted_date = date_obj.isoformat()
    
    # This step is necessary to save the date outside of the function
    global valid_date
    if (valid_date == ""):
        valid_date = f"{formatted_date}Z"
    
    print("ExtremeDT data available=>" + formatted_date)

## Listener configuration
This helper function facilitates the configuration of the listener

In [ ]:
def create_hist_listener():
    """
    Creates and returns a listener configuration.
    """

    trigger = {
        "type": TRIGGER_TYPE,
        "function": triggered_function,
    }  # Define the trigger for the listener
    
    # Return the complete listener configuration
    return {"event": LISTENER_EVENT, "request": REQUEST, "triggers": [trigger]}

## Listening for notifications
The Notification Manager is configured to list past recent events. after the listening process is finished (it takes around 30 seconds), the notebooks execution will resume, with the valid date set in the appropriate variable.

In [ ]:
listener = create_hist_listener()  # Create listener configuration
listeners_config = {"listeners": [listener]}  # Define listeners configuration
config = user_config.UserConfig(**CONFIG)
print("Configuration loaded.")
nmh = NotificationManager()  # Initialize the NotificationManager
print(f"Notification Manager created, starting listening...")

nmh.listen(
    listeners=listeners_config, from_date=START_DATE, to_date=END_DATE, config=config
)  # Start listening

# Search For Products Inside The DT Collection

This cell focuses on using EODAG to drill into the collection, using its ID and a several search parameters. A non-empty list of features is expected.


In [ ]:
# SEARCH FOR PRODUCTS INSIDE A DATASET

from datetime import datetime
from eodag import setup_logging, SearchResult
from random import randrange
import copy

setup_logging(1)  # 0: nothing, 1: only progress bars, 2: INFO, 3: DEBUG

collection_search_result = dag.search(
    productType="EO.ECMWF.DAT.DT_EXTREMES",
    start=valid_date,
    end=valid_date,
    **{"class": "d1"},     # fixed 
    dataset="extremes-dt", # fixed extremes-dt access
    expver="0001",         # fixed experiment version 
    stream="oper",         # fixed operation
    type="fc",             # fixed forecasted fields
    levtype="sfc",         # Surface fields (levtype=sfc), Height level fields (levtype=hl), Pressure level fields (levtype=pl), Model Level (Levtype=ml)
    step="0",              # Forcast step hourly (1..96)
    param="31"             # Surface Pressure parameter
    )

entryWrite(
    f"Parameterized search of product inside 'EO.ECMWF.DAT.DT_EXTREMES' collection)",
    "A non-empty list of features present in the selected dataset",
    "List size: " + str(len(collection_search_result)),
    "PASS" if (len(collection_search_result) != 0 ) else "FAIL"
)

print(collection_search_result)
print(collection_search_result[0].search_kwargs)

print("Entry added to the report")

print("Collection drilldown finished")


# Download Of A Randomly Chosen Feature

A random feature is selected from the previously chosen collection and downloaded.

In [ ]:
import os
from eodag.api.product._product import EOProduct

print("File initial location: " + collection_search_result[0].location)

productPath = dag.download(collection_search_result[0])
product_exists = os.path.exists(productPath) and len(os.listdir(productPath)) != 0

if product_exists:
    productPath = f"{productPath}/{os.listdir(productPath)[0]}"

entryWrite(
    "EoProductObj.download()",
    "A new file is created with the downloaded content",
    "File was created: " + str(product_exists) + (";" if product_exists == False else f"; file path: {productPath}"),
    "PASS" if (product_exists == True ) else "FAIL"
)
print("Entry added to the report")
    
